In [31]:
import datetime as dt
import numpy as np

In [32]:
dates = [dt.datetime(2015,1,1),dt.datetime(2015,7,1),dt.datetime(2016,1,1)]

print((dates[1] - dates[0]).days/365)
print((dates[2]-dates[1]).days/365)


0.4958904109589041
0.5041095890410959


In [33]:
# Get year fractions out of a list of dates

def get_year_deltas(date_list , day_count = 365):
    """ Return array of floats with day deltas in years with initial value normalized to zero
        Parameters:
        date_list: list or array collection of datetime objects
        day_count : number of days of a year (to account for different conventions)
    
    """

    return np.array([(date-date_list[0]).days/day_count for date in date_list])

In [35]:
def get_discount_factors(date_list , const_rate):
    """
    get a list of discount factors when given a list of dates and a constant risk free rate
    """

    return np.exp(-const_rate*get_year_deltas(date_list))

print(get_discount_factors(dates,0.05))

[1.         0.97551034 0.95122942]


In [53]:
# Class for risk neutral discounting with constant short rate 

class constant_short_rate():

    """
    Class for short rate discounting 

    Attributes---

    name : string ---- name of the object
    short_rate : float ----- const rate for discounting 

    Methods ------

    get_discount_factors : get_discount_factors when given a list of datetime objects or year fractions
    """

    def __init__(self,name,short_rate):
        self.name = name 
        self.short_rate = short_rate
        
    def get_discount_factors(self,date_list):
        discount_factors = np.exp(-self.short_rate*get_year_deltas(date_list))

        return np.array((date_list,discount_factors)).T
    




In [56]:
csr = constant_short_rate("r",0.05)
csr

In [63]:
class market_environmnet():
    """
    Class to model a market environment relevant for valuation

    Attributes ------
    name : name of market environment
    pricing_date : date of market environment
    
    Methods:
    add_constant: adds a model parameter
    get_constant: gets a model parameter
    add_list : adds a list
    get_list : gets a lsit
    add_curve : adds a curve
    get_curve : gets a curve

    """

    def __init__(self,name,pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}

    def add_constant(self,key,constant):
        self.constants[key] = constant

    def get_constant(self,key):
        return self.constants[key]
    
    def add_list(self,key,list):
        self.lists[key] = list

    def get_list(self,key):
        return self.lists[key]

    def add_curve(self,key,curve):
        self.curves[key] = curve

    def get_curve(self,key):
        return self.curves[key]
    
    def add_environment(self,env):
        for key in env.constants:
            self.constants[key] = env.constants[key]

        for key in env.lists:
            self.lists[key] = env.lists[key]

        for key in env.curves:
            self.curves[key] = env.curves[key]


In [64]:
me_1 = market_environmnet("me_1",dt.datetime(2015,1,1))

In [65]:
me_1.add_list("stocks",["AAPL","MSFT","TSLA"])
me_1.get_list("stocks")

['AAPL', 'MSFT', 'TSLA']

In [66]:
me_2 = market_environmnet("me_2", dt.datetime(2015,1,1))
me_2.add_constant("volatility" , 0.2)
me_2.add_curve("short_rate" , csr )
me_2.get_curve("short_rate")
me_1.add_environment(me_2)


In [69]:
me_1.curves

{'short_rate': <__main__.constant_short_rate at 0x25b2323b310>}

In [ ]:
# Generic Financial Model simulation class

class simulation_class():
    """
    Providing base methods for simulation classes

    Attributes -- 

    name : name of the object
    mar_env : instance of market_environment data for simulation
    corr : Boolean ,  True if correlated with other model object

    Methods:
    generate time_grid: returns time grid for simulation
    get_instrument_values:returns current instrument values

    """

    def __init__(self,name,mark_env,corr):
        try:
            self.name = name
            self.pricing_date = mark_env.pricing_date
            self.initial_value = mark_env.get_constant("initial_value")
            self.volatility = mark_env.get_constant("volatility")
            self.finaldate = mark_env.get_constant("final_date")
            self.currency = mark_env.get_constant("currency")
            self.frequency = mark_env.get_constant("frequency")
            self.paths = mark_env.get_constant("paths")
            self.discount_curve = mark_env.get_curve("discount_curve")


        except:
            print( "Error parsing market environmnet" )